## **Clone Git**

---


Clone git agar dapat load data langsung dari git repository. Dataset yang digunakan didapat kan dari kaggle: https://www.kaggle.com/datasets/sudalairajkumar/cryptocurrencypricehistory?select=coin_Ethereum.csv

In [ ]:
!git clone https://github.com/ziszz/ethereum-price-predict.git

## **Import Library**

---


Import beberapa library yang diperlukan untuk analisa data, visualisasi data dan melatih model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

## **Data Loading**

---



In [ ]:
data_path = '/content/ethereum-price-predict/dataset/coin_Ethereum.csv'

df = pd.read_csv(data_path)
df

## **Exploratory Data** 

---

### **Variabel-variabel pada dataset adalah sebagai berikut:**

* Name: Nama mata uang kripto
* Symbol: Simbol mata uang kripto
* Date: Tanggal pencatatan data
* High : Harga tertinggi pada hari tertentu
* Low : Harga terendah pada hari tertentu
* Open : Harga pembukaan pada hari tertentu
* Close : Harga penutupan pada hari tertentu
* Volume : Volume transaksi pada hari tertentu
* Mastercap : Kapitalisasi pasar dalam USD

### **Mengecek informasi pada data**

Melakukan pengecekan apakah terdapat missing value pada data dan kesesuaian tipe data

In [ ]:
df.info()

Dapat dilihat bahwa tidak terdapat missing value pada dataset

### **Informasi Statistik**
Data di atas memiliki beberapa informasi statistik pada masing-masing kolom, antara lain:

* count adalah jumlah sampel pada data.
* mean adalah nilai rata-rata.
* std adalah standar deviasi.
* min yaitu nilai minimum setiap kolom.
* 25% adalah kuartil pertama. Kuartil adalah nilai yang menandai batas interval dalam empat bagian sebaran yang sama.
* 50% adalah kuartil kedua, atau biasa juga disebut median (nilai tengah).
* 75% adalah kuartil ketiga.
* Max adalah nilai maksimum





In [ ]:
df.describe()

Dapat dilihat pada visualisasi data dibawah. Terdapat banyak data outlier pada masing masing kolom data.

In [ ]:
numeric_features = df.select_dtypes(include=np.number).columns.tolist()
plt.figure(figsize=(15, 8))

for i, col in enumerate(numeric_features):
  plt.subplot(3,3,i+1)
  df.boxplot(column=col)

Terdapat beberapa teknik untuk mengatasi outlier pada data. Pada project ini akan menerapkan teknik IQR Method yaitu dengan menghapus data yang berada diluar interquartile range. Interquartile merupakan range diantara kuartil pertama(25%) dan kuartil ketiga(75%).

In [ ]:
Q1 = df.quantile(.25)
Q3 = df.quantile(.75)

IQR = Q3 - Q1

bot_treshold = Q1 - 1.5 * IQR
top_treshold = Q3 + 1.5 * IQR

df = df[~((df < bot_treshold) | (df > top_treshold)).any(axis=1)]
df.shape

### **Univariate Analysis**
Karena target prediksi dari dataset ini ada pada fitur Close_Price yang merupakan harga crypto coin Ethereum, jadi hanya fokus menganalisis korelasi data pada feature tersebut. Dari hasil visualisasi data dibawah dapat disimpulkan bahwa peningkatan harga crypto coin ethereum sebanding dengan penurunan jumlah sampel data.

In [ ]:
df.hist(bins=50, figsize=(15, 10))
plt.show()

### **Multivariate Analysis**
Jika di lihat dari visualisasi data dibawah. Fitur Close pada sumbu y memiliki korelasi dengan data pada fitur High, Low, Open, dan Marketcap. Korelasi yang terdapat pada data-data tersebut merupakan korelas yang tinggi, sedangkan untuk fitur Volume terlihat memiliki korelasi yang cukup lemah karena sebaran datanya tidak membentuk pola.

In [ ]:
sns.pairplot(df, diag_kind = 'kde')
plt.show()

Untuk lebih jelasnya dapat dilihat melalui visualisasi dibawah yang menunjukkan skor korelasi di tiap fitur dengan fitur Close. Pada fitur High, Low, Open dan Marketcap memiliki skor korelasi yang terbilang tinggi yaitu di atas 0.9. Sedangkan pada fitur Volume memiliki skor korelasi yang cukup rendah yaitu 0.38. Sehingga fitur Volume ini dapat didrop dari dataset.

In [ ]:
plt.figure(figsize=(10, 8))
correlation_matrix = df.corr().round(2)
 
sns.heatmap(data=correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title("Correlation Matrix untuk Fitur Numerik ", size=20)

In [ ]:
df.drop(['Volume'], axis=1, inplace=True)
df

## **Data Preparation**

---

### **Menghapus data yang tidak diperlukan dan merubah nama column**

Kolom data seperti (SNo, Name, Symbol, Date, Marketcap) tidak diperlukan untuk pelatihan, karena data tersebut akan mengganggu model dalam mempelajari data. Karena isi dari data tersebut tidak memiliki value yang berarti untuk dipelajari oleh model. Lalu, mengubah nama kolom High, Low, Open, Close menjadi nama kolom yang dapat lebih dipahami.


In [ ]:
unused_columns = ['SNo', 'Name', 'Symbol', 'Date', 'Marketcap']
renamed_columns = {'High': 'High_Price', 'Low': 'Low_Price', 
                   'Open': 'Open_Price', 'Close': 'Close_Price'}

df.drop(unused_columns, axis=1, inplace=True)
df.rename(columns=renamed_columns, inplace=True)
df

### **Split dataset**
Membagi dataset menjadi data training dan data testing

In [ ]:
x = df.drop(['Close_Price'], axis=1).values
y = df['Close_Price'].values

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=1)

print(f'Total # of sample in whole dataset: {len(x)}')
print(f'Total # of sample in train dataset: {len(x_train)}')
print(f'Total # of sample in test dataset: {len(x_test)}')

### **Normalisasi data**
Melakukan transformasi pada data fitur fitur yang akan dipelajari oleh model menggunakan library MinMaxScaler. MinMaxScaler mentransformasikan fitur dengan menskalakan setiap fitur ke rentang tertentu. Library ini menskalakan dan mentransformasikan setiap fitur secara individual sehingga berada dalam rentang yang diberikan pada set pelatihan, pada library ini memiliki range default antara nol dan satu.

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)

In [ ]:
models = pd.DataFrame(columns=['train_mse', 'test_mse'],
                      index=['KNN', 'RandomForest', 'SVR'])

## **Model Development**

---


### **Tuning Hyperparameters**
Melakukan tuning hyperparameters untuk mendapatkan parameter dengan performa terbaik pada model. Untuk melakukan tuning hyperparameter pada proyek ini menggunakan teknik Grid search. Grid search memungkinkan untuk menguji beberapa parameter sekaligus pada sebuah model. Dengan menerapkan teknik ini kita dapat melihat performa model terbaik dengan parameter tertentu. Hasil dari grid search terkadang berbeda setelah kode dijalankan ulang.


In [ ]:
# svr = SVR()
# parameters = {
#     'kernel': ['rbf'],
#     'C':     [1000, 10000, 100000],
#     'gamma': [0.3, 0.03, 0.003]
# }

# grid_search = GridSearchCV(
#     svr, 
#     parameters,
#     cv=5, 
#     verbose=1,
#     n_jobs=6,
# )

# grid_search.fit(x_train, y_train)

# print(grid_search.best_params_)
# print(grid_search.best_score_)

In [ ]:
# knn = KNeighborsRegressor()
# parameters =  {
#     'n_neighbors': range(1, 25),
# }

# grid_search = GridSearchCV(
#   knn, 
#   parameters, 
#   cv=5,
#   verbose=1, 
#   n_jobs=6,
# )

# grid_search.fit(x_train, y_train)

# print(grid_search.best_params_)
# print(grid_search.best_score_)

In [ ]:
# rf = RandomForestRegressor()
# parameters =  {
#     'n_estimators': range(1, 10),
#     'max_depth': [16, 32, 64],
# }

# grid_search = GridSearchCV(
#   rf, 
#   parameters, 
#   cv=5,
#   verbose=1,
#   n_jobs=6,
# )
# grid_search.fit(x_train, y_train)

# print(grid_search.best_params_)
# print(grid_search.best_score_)

### **Training Model**
**Support Vector Machine**

In [ ]:
svr = SVR(C=100000, gamma=0.003, kernel='rbf')                          
svr.fit(x_train, y_train)

**K-Nearest Neighbours**

In [ ]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(x_train, y_train)

**Random Forest**

In [ ]:
rf = RandomForestRegressor(n_estimators=6, max_depth=16)
rf.fit(x_train, y_train)

### **Model Evaluation**

In [ ]:
x_test = scaler.transform(x_test)

In [ ]:
model_dict = {'KNN': knn, 'RandomForest': rf, 'SVR': svr}

for name, model in model_dict.items():
  models.loc[name, 'train_mse'] = mean_squared_error(y_true=y_train, y_pred=model.predict(x_train))
  models.loc[name, 'test_mse'] = mean_squared_error(y_true=y_test, y_pred=model.predict(x_test)) 

models

In [ ]:
fig, ax = plt.subplots()
models.sort_values(by='test_mse', ascending=False).plot(kind='barh', ax=ax, zorder=3)
ax.grid(zorder=0)

In [ ]:
test_data = x_test.copy()
predictions = {'y_true':y_test}
for name, model in model_dict.items():
  predictions['prediction_' + name] = model.predict(test_data)
 
predictions = pd.DataFrame(predictions)
predictions

In [ ]:
predictions = predictions.tail(10)
predictions.plot(kind='bar',figsize=(16,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

Dari ketiga model yang digunakan. Terlihat bahwa hampir semua model yang digunakan memiliki hasil prediksi mendekati dengan hasil yang sebenarnya.